<a href="https://colab.research.google.com/github/simoneseverini/automated-discovery-site/blob/main/Matula_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!/usr/bin/env python3
"""
==============================================================================
VERIFICATION SCRIPT FOR:
"The Matula metric on the natural numbers" by Simone Severini

This script independently verifies every computational claim, bound, table
entry, and numerically checkable theorem statement in the paper. It is
designed to run as a Google Colab notebook (copy cells between separator
comments) or as a standalone Python script.

Requirements: Python 3.8+, scipy (for Hungarian algorithm on larger instances)
Install:  pip install scipy

Repository: [GitHub URL]
==============================================================================
"""

# ===========================================================================
# CELL 1: CORE INFRASTRUCTURE
# ===========================================================================

import math
import sys
from functools import lru_cache
from itertools import permutations
from collections import defaultdict

print("=" * 72)
print("VERIFICATION SCRIPT")
print("'The Matula metric on the natural numbers' — S. Severini")
print("=" * 72)
print()

# ---- Global counters for pass/fail ----
_pass_count = 0
_fail_count = 0
_warn_count = 0

def CHECK(condition, description, detail=""):
    """Assert a verifiable claim. Logs pass/fail."""
    global _pass_count, _fail_count
    if condition:
        _pass_count += 1
        print(f"  ✓ {description}")
    else:
        _fail_count += 1
        print(f"  ✗ FAIL: {description}")
        if detail:
            print(f"         {detail}")

def WARN(description):
    global _warn_count
    _warn_count += 1
    print(f"  ⚠ WARNING: {description}")

def SECTION(title):
    print()
    print("-" * 72)
    print(f"  {title}")
    print("-" * 72)

# ===========================================================================
# CELL 2: PRIME NUMBER UTILITIES
# ===========================================================================

def sieve_primes(n):
    """Return list of primes up to n via Sieve of Eratosthenes."""
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False
    for i in range(2, int(n**0.5) + 1):
        if is_prime[i]:
            for j in range(i * i, n + 1, i):
                is_prime[j] = False
    return [i for i in range(2, n + 1) if is_prime[i]]

PRIMES = sieve_primes(200000)
PRIME_SET = set(PRIMES)
PRIME_INDEX = {p: i + 1 for i, p in enumerate(PRIMES)}  # 1-indexed


def nth_prime(k):
    """Return the k-th prime (1-indexed: p_1=2, p_2=3, ...)."""
    return PRIMES[k - 1]


def prime_index(p):
    """Return k such that p = p_k."""
    return PRIME_INDEX[p]


def factorize(n):
    """Return list of (prime, exponent) pairs in the canonical factorisation."""
    if n == 1:
        return []
    factors = []
    temp = n
    for p in PRIMES:
        if p * p > temp:
            break
        if temp % p == 0:
            e = 0
            while temp % p == 0:
                e += 1
                temp //= p
            factors.append((p, e))
    if temp > 1:
        factors.append((temp, 1))
    return factors


def Omega(n):
    """Number of prime factors counted with multiplicity."""
    return sum(e for _, e in factorize(n))


# ===========================================================================
# CELL 3: MATULA TREE REPRESENTATION
# ===========================================================================
#
# A tree is represented as a tuple of children-subtrees, each itself a tuple.
# The single-vertex tree (T(1)) is the empty tuple ().
# Trees are stored in sorted canonical form for uniqueness.

@lru_cache(maxsize=None)
def matula_tree(n):
    """
    Compute the Matula tree T(n) as a canonical nested tuple.

    T(1) = ()  (single vertex)
    T(n) for n >= 2: root with children T(i_j) repeated a_j times,
    where n = p_{i_1}^{a_1} * ... * p_{i_k}^{a_k}.
    """
    if n == 1:
        return ()
    children = []
    for p, e in factorize(n):
        idx = prime_index(p)
        subtree = matula_tree(idx)
        children.extend([subtree] * e)
    return tuple(sorted(children))


@lru_cache(maxsize=None)
def tree_size(t):
    """Number of vertices in tree t (including root)."""
    return 1 + sum(tree_size(c) for c in t)


def f(n):
    """The vertex-count function f(n) = |T(n)|."""
    return tree_size(matula_tree(n))


# ===========================================================================
# CELL 4: TREE EDIT DISTANCE (EXACT, UNORDERED)
# ===========================================================================
#
# For unordered unlabelled rooted trees, the edit distance is:
#   δ(S, T) = |S| + |T| - 2γ(S, T)
# where γ(S, T) is the maximum common embedded subtree, computed via
# maximum-weight bipartite matching on children.

def _min_cost_matching_brute(cost_matrix):
    """Exact minimum-cost perfect matching via brute force (n ≤ 8)."""
    n = len(cost_matrix)
    if n == 0:
        return 0
    best = float('inf')
    for perm in permutations(range(n)):
        cost = sum(cost_matrix[i][perm[i]] for i in range(n))
        best = min(best, cost)
    return best


def _min_cost_matching_hungarian(cost_matrix):
    """Minimum-cost perfect matching via scipy Hungarian algorithm."""
    from scipy.optimize import linear_sum_assignment
    import numpy as np
    arr = np.array(cost_matrix, dtype=float)
    row_ind, col_ind = linear_sum_assignment(arr)
    return int(arr[row_ind, col_ind].sum())


def min_cost_matching(cost_matrix):
    """Minimum-cost perfect matching for a square matrix."""
    n = len(cost_matrix)
    if n == 0:
        return 0
    if n <= 8:
        return _min_cost_matching_brute(cost_matrix)
    return _min_cost_matching_hungarian(cost_matrix)


@lru_cache(maxsize=None)
def mces(s, t):
    """
    Maximum common embedded subtree size γ(S, T).
    Roots are always matched (both trees are rooted).
    γ(S, T) = 1 + max bipartite matching on children.
    """
    cs = list(s)  # children of s
    ct = list(t)  # children of t
    m, n = len(cs), len(ct)

    if m == 0 or n == 0:
        return 1  # only roots can match

    k = max(m, n)
    # Build weight matrix (want to maximise total mces over matched pairs)
    # Pad to k × k with zeros for unmatched children
    weights = [[0] * k for _ in range(k)]
    for i in range(m):
        for j in range(n):
            weights[i][j] = mces(cs[i], ct[j])

    # Negate for min-cost matching
    neg = [[-w for w in row] for row in weights]
    max_match = -min_cost_matching(neg)

    return 1 + max_match


@lru_cache(maxsize=None)
def ted(s, t):
    """Tree edit distance δ(S, T) = |S| + |T| - 2γ(S, T)."""
    return tree_size(s) + tree_size(t) - 2 * mces(s, t)


def d(m, n):
    """The Matula metric d(m, n) = δ(T(m), T(n))."""
    return ted(matula_tree(m), matula_tree(n))


# ===========================================================================
# CELL 5: VERIFY EXAMPLE 2.1 — Small Matula trees
# ===========================================================================

SECTION("EXAMPLE 2.1: Matula trees T(n) for n = 1, ..., 8")

# Paper claims (Table in Example 2.1):
example_2_1 = {
    1: (1, "single vertex"),
    2: (2, "root + one leaf"),
    3: (3, "path of length 2"),
    4: (3, "root + two leaves (star S_2)"),
    5: (4, "path of length 3"),
    6: (4, "T(1) and T(2) attached to root"),
    7: (4, "root + star S_2"),
    8: (4, "root + three leaves (star S_3)"),
}

for n, (claimed_size, desc) in example_2_1.items():
    actual_size = f(n)
    CHECK(actual_size == claimed_size,
          f"|T({n})| = {claimed_size} ({desc})",
          f"got |T({n})| = {actual_size}")

# Verify tree shapes match descriptions
CHECK(matula_tree(1) == (), "T(1) is single vertex ()")
CHECK(matula_tree(2) == ((),), "T(2) = root + one leaf")
CHECK(matula_tree(3) == (((),),), "T(3) = path: root → child → leaf")
CHECK(matula_tree(4) == ((), ()), "T(4) = root + two leaves")
CHECK(matula_tree(5) == ((((),),),), "T(5) = path of 4 vertices")
CHECK(matula_tree(8) == ((), (), ()), "T(8) = root + three leaves (star)")

# T(6) = p_1 * p_2 = root with children T(1)=() and T(2)=((),)
CHECK(matula_tree(6) == ((), ((),)), "T(6) = root with T(1) and T(2) as children")

# T(7) = p_4: planted tree with T(4) as subtree
# T(4) = ((), ()) so T(7) = (((), ()),)  -> root + S_2
CHECK(matula_tree(7) == (((), ()),), "T(7) = planted S_2")


# ===========================================================================
# CELL 6: VERIFY PROPOSITION 3.1 — Metric axioms
# ===========================================================================

SECTION("PROPOSITION 3.1: d is a metric on N")

N_METRIC = 30
print(f"  Checking metric axioms for all m, n, k in [1, {N_METRIC}]...")

# (i) d(m, m) = 0
for m in range(1, N_METRIC + 1):
    if d(m, m) != 0:
        CHECK(False, f"d({m},{m}) = 0", f"got {d(m,m)}")
        break
else:
    CHECK(True, f"d(m,m) = 0 for all m in [1, {N_METRIC}]")

# (ii) d(m, n) = 0 => m = n (already guaranteed by bijection)
for m in range(1, N_METRIC + 1):
    for n in range(m + 1, N_METRIC + 1):
        if d(m, n) == 0:
            CHECK(False, f"d(m,n) > 0 for m ≠ n", f"d({m},{n}) = 0")
            break
    else:
        continue
    break
else:
    CHECK(True, f"d(m,n) > 0 for all m ≠ n in [1, {N_METRIC}]")

# (iii) Symmetry
sym_ok = True
for m in range(1, N_METRIC + 1):
    for n in range(m + 1, N_METRIC + 1):
        if d(m, n) != d(n, m):
            CHECK(False, "Symmetry", f"d({m},{n})={d(m,n)} ≠ d({n},{m})={d(n,m)}")
            sym_ok = False
            break
    if not sym_ok:
        break
if sym_ok:
    CHECK(True, f"d(m,n) = d(n,m) for all m,n in [1, {N_METRIC}]")

# (iv) Triangle inequality
tri_ok = True
tri_violations = 0
for m in range(1, 21):  # smaller range for speed
    for n in range(1, 21):
        for k in range(1, 21):
            if d(m, k) > d(m, n) + d(n, k):
                CHECK(False, "Triangle inequality",
                      f"d({m},{k})={d(m,k)} > d({m},{n})+d({n},{k})="
                      f"{d(m,n)+d(n,k)}")
                tri_ok = False
                tri_violations += 1
                if tri_violations >= 3:
                    break
        if not tri_ok and tri_violations >= 3:
            break
    if not tri_ok and tri_violations >= 3:
        break
if tri_ok:
    CHECK(True, "Triangle inequality d(m,k) ≤ d(m,n) + d(n,k) for all m,n,k in [1, 20]")


# ===========================================================================
# CELL 7: VERIFY PROPOSITION 4.1 — Recursion for f
# ===========================================================================

SECTION("PROPOSITION 4.1: f(n) = 1 + Σ a_j f(i_j)")

CHECK(f(1) == 1, "f(1) = 1")

rec_ok = True
for n in range(2, 201):
    factors = factorize(n)
    rhs = 1 + sum(a * f(prime_index(p)) for p, a in factors)
    if f(n) != rhs:
        CHECK(False, f"f-recursion at n={n}", f"f({n})={f(n)}, rhs={rhs}")
        rec_ok = False
        break
if rec_ok:
    CHECK(True, "f(n) = 1 + Σ a_j f(i_j) verified for all n in [2, 200]")


# ===========================================================================
# CELL 8: VERIFY PROPOSITION 4.3 — Distance from 1
# ===========================================================================

SECTION("PROPOSITION 4.3: d(1, n) = f(n) - 1")

dist1_ok = True
for n in range(1, 201):
    if d(1, n) != f(n) - 1:
        CHECK(False, f"d(1,{n}) = f({n})-1", f"d(1,{n})={d(1,n)}, f({n})-1={f(n)-1}")
        dist1_ok = False
        break
if dist1_ok:
    CHECK(True, "d(1, n) = f(n) - 1 for all n in [1, 200]")


# ===========================================================================
# CELL 9: VERIFY THEOREM 4.4 — Sharp bound on f(n)
# ===========================================================================

SECTION("THEOREM 4.4: f(n) ≤ 1 + (3/log 5) log n, equality iff n = 5^k")

alpha = 3.0 / math.log(5)
print(f"  α = 3/log(5) = {alpha:.10f}")
print(f"  5^(1/3) = {5**(1/3):.10f}")
print()

# (a) Verify the Claim: f(i) ≤ α log p_i for all i
SECTION("  CLAIM: f(i) ≤ α log(p_i) for all i ≥ 1")

claim_ok = True
claim_equality = []
for i in range(1, 1001):
    fi = f(i)
    bound = alpha * math.log(nth_prime(i))
    if fi > bound + 1e-9:
        CHECK(False, f"Claim at i={i}", f"f({i})={fi}, α·log(p_{i})={bound:.6f}")
        claim_ok = False
        break
    if abs(fi - bound) < 1e-9:
        claim_equality.append(i)
if claim_ok:
    CHECK(True, "f(i) ≤ α log(p_i) for all i in [1, 1000]")
    CHECK(claim_equality == [3],
          f"Equality f(i) = α log(p_i) holds only at i = 3",
          f"equality at i ∈ {claim_equality}")

# (b) Verify p_i/i ≥ 5^{1/3} for all i ≥ 4
SECTION("  KEY INEQUALITY: p_i/i ≥ 5^(1/3) for i ≥ 4")

threshold = 5 ** (1.0 / 3)
min_ratio = float('inf')
min_ratio_i = -1
ratio_ok = True
for i in range(4, 10001):
    ratio = nth_prime(i) / i
    if ratio < min_ratio:
        min_ratio = ratio
        min_ratio_i = i
    if ratio < threshold - 1e-12:
        CHECK(False, f"p_i/i ≥ 5^(1/3) at i={i}",
              f"p_{i}/i = {nth_prime(i)}/{i} = {ratio:.6f} < {threshold:.6f}")
        ratio_ok = False
        break
if ratio_ok:
    CHECK(True, f"p_i/i ≥ 5^(1/3) for all i in [4, 10000]")
    print(f"    Minimum ratio: p_{min_ratio_i}/{min_ratio_i} = "
          f"{nth_prime(min_ratio_i)}/{min_ratio_i} = {min_ratio:.6f}")
    print(f"    5^(1/3) = {threshold:.6f}")

# (c) Base cases
SECTION("  BASE CASES of the Claim")
CHECK(f(1) == 1 and 1 <= alpha * math.log(2),
      f"f(1) = 1 ≤ α log 2 = {alpha * math.log(2):.4f}")
CHECK(f(2) == 2 and 2 <= alpha * math.log(3),
      f"f(2) = 2 ≤ α log 3 = {alpha * math.log(3):.4f}")
CHECK(abs(f(3) - alpha * math.log(5)) < 1e-9,
      f"f(3) = 3 = α log 5 = {alpha * math.log(5):.4f} (equality)")

# (d) Main bound: f(n) ≤ 1 + α log n for all n ≥ 1
SECTION("  MAIN BOUND: f(n) ≤ 1 + α log n")

bound_ok = True
for n in range(1, 10001):
    fn = f(n)
    bound_val = 1 + alpha * math.log(n) if n >= 2 else 1
    if fn > bound_val + 1e-9:
        CHECK(False, f"f({n}) ≤ 1 + α log({n})",
              f"f({n})={fn}, bound={bound_val:.6f}")
        bound_ok = False
        break
if bound_ok:
    CHECK(True, "f(n) ≤ 1 + α log n for all n in [1, 10000]")

# (e) Equality exactly at powers of 5
SECTION("  EQUALITY: f(n) = 1 + α log n iff n = 5^k")

# First check powers of 5
for k in range(0, 10):
    n = 5 ** k
    fn = f(n)
    expected = 3 * k + 1
    bound_val = 1 + alpha * math.log(n) if n >= 2 else 1.0
    CHECK(fn == expected,
          f"f(5^{k}) = f({n}) = {expected}",
          f"got {fn}")
    if n >= 2:
        CHECK(abs(fn - bound_val) < 1e-9,
              f"f(5^{k}) = 1 + α log(5^{k}) = {bound_val:.6f} (equality)",
              f"f = {fn}, bound = {bound_val:.6f}")

# Check no other n achieves equality
equality_others = []
for n in range(2, 5001):
    fn = f(n)
    bound_val = 1 + alpha * math.log(n)
    if abs(fn - bound_val) < 1e-6:
        # Check if n is a power of 5
        temp = n
        while temp > 1 and temp % 5 == 0:
            temp //= 5
        if temp != 1:
            equality_others.append(n)
CHECK(len(equality_others) == 0,
      "No n in [2, 5000] that is NOT a power of 5 achieves equality",
      f"violations at n ∈ {equality_others}")


# ===========================================================================
# CELL 10: VERIFY COROLLARY 4.5 — Limsup
# ===========================================================================

SECTION("COROLLARY 4.5: limsup f(n)/log(n) = 3/log 5")

# The bound is f(n) ≤ 1 + α log n, so f(n)/log n ≤ α + 1/log n.
# For the limsup, we need:
# (a) Along n = 5^k: f(5^k)/log(5^k) → α as k → ∞
# (b) For all n: f(n)/log n ≤ α + 1/log n (so limsup ≤ α)

# Check (a): the sequence 5^k achieves the limsup
for k in range(1, 10):
    n = 5 ** k
    ratio = f(n) / math.log(n)
    deviation = abs(ratio - alpha)
    # f(5^k) = 1 + 3k, log(5^k) = k log 5, so ratio = (1 + 3k)/(k log 5) → 3/log 5
    expected_ratio = (1 + 3 * k) / (k * math.log(5))
    CHECK(abs(ratio - expected_ratio) < 1e-9,
          f"f(5^{k})/log(5^{k}) = {ratio:.6f} (→ α = {alpha:.6f} as k → ∞)")

# Check (b): no n in [2, 10000] has f(n)/log(n) > α + 1/log(n) + ε
bound_exceeded = False
for n in range(2, 10001):
    ratio = f(n) / math.log(n)
    upper = alpha + 1.0 / math.log(n)
    if ratio > upper + 1e-9:
        CHECK(False, f"f(n)/log(n) ≤ α + 1/log(n) at n={n}",
              f"ratio = {ratio:.6f}, bound = {upper:.6f}")
        bound_exceeded = True
        break
if not bound_exceeded:
    CHECK(True, "f(n)/log(n) ≤ α + 1/log(n) for all n in [2, 10000] "
          "(confirms limsup ≤ α)")


# ===========================================================================
# CELL 11: VERIFY LEMMA 5.1 — Isometry Lemma
# ===========================================================================

SECTION("LEMMA 5.1 (ISOMETRY): d(p_i, p_j) = d(i, j)")

iso_ok = True
iso_count = 0
for i in range(1, 51):
    for j in range(i + 1, 51):
        pi = nth_prime(i)
        pj = nth_prime(j)
        d_primes = d(pi, pj)
        d_indices = d(i, j)
        if d_primes != d_indices:
            CHECK(False, f"d(p_{i}, p_{j}) = d({pi}, {pj}) = d({i},{j})",
                  f"d({pi},{pj}) = {d_primes} ≠ d({i},{j}) = {d_indices}")
            iso_ok = False
            break
        iso_count += 1
    if not iso_ok:
        break
if iso_ok:
    CHECK(True, f"d(p_i, p_j) = d(i, j) for all 1 ≤ i < j ≤ 50 "
          f"({iso_count} pairs checked)")


# ===========================================================================
# CELL 12: VERIFY COROLLARY 5.3 — Iterated self-similarity
# ===========================================================================

SECTION("COROLLARY 5.3: d(p^(k)(i), p^(k)(j)) = d(i, j)")

def iterated_prime(n, k):
    """Compute p^(k)(n): k-fold iterated prime indexing."""
    result = n
    for _ in range(k):
        result = nth_prime(result)
    return result

iter_ok = True
for k in range(0, 5):
    for i in range(1, 8):
        for j in range(i + 1, 8):
            pki = iterated_prime(i, k)
            pkj = iterated_prime(j, k)
            if pki <= 200 and pkj <= 200:
                d_iterated = d(pki, pkj)
                d_base = d(i, j)
                if d_iterated != d_base:
                    CHECK(False,
                          f"d(p^({k})({i}), p^({k})({j})) = d({i},{j})",
                          f"d({pki}, {pkj}) = {d_iterated} ≠ {d_base}")
                    iter_ok = False
if iter_ok:
    CHECK(True, "d(p^(k)(i), p^(k)(j)) = d(i,j) for k=0..4, i,j in [1,7]")


# ===========================================================================
# CELL 13: VERIFY PROPOSITION 5.4 — The spine
# ===========================================================================

SECTION("PROPOSITION 5.4: Spine distances d(a_k, a_ℓ) = |k - ℓ|")

# Construct the spine: a_0 = 1, a_{k+1} = p_{a_k}
spine = [1]
n = 1
for _ in range(7):
    n = nth_prime(n)
    spine.append(n)

print(f"  Spine: {spine}")

spine_ok = True
for k in range(len(spine)):
    for ell in range(k, len(spine)):
        if spine[ell] <= 200:
            actual = d(spine[k], spine[ell])
            expected = abs(k - ell)
            if actual != expected:
                CHECK(False, f"d(a_{k}, a_{ell}) = |{k} - {ell}|",
                      f"d({spine[k]}, {spine[ell]}) = {actual} ≠ {expected}")
                spine_ok = False
                break
    if not spine_ok:
        break
if spine_ok:
    CHECK(True, "d(a_k, a_ℓ) = |k - ℓ| for all computed spine elements")

# Verify T(a_k) is a path of k+1 vertices
for k in range(len(spine)):
    CHECK(f(spine[k]) == k + 1,
          f"|T(a_{k})| = |T({spine[k]})| = {k + 1}",
          f"got {f(spine[k])}")


# ===========================================================================
# CELL 14: VERIFY THEOREM 6.1 — Diameter bounds
# ===========================================================================

SECTION("THEOREM 6.1: c_1 log N ≤ D(N) ≤ c_2 log N")

c1 = 1.0 / math.log(2)
c2 = 6.0 / math.log(5)

print(f"  c_1 = 1/log 2 = {c1:.6f}")
print(f"  c_2 = 6/log 5 = {c2:.6f}")
print()

# Compute D(N) for various N
diameter_claimed = {5: 3, 10: 4, 15: 5, 20: 6, 30: 7, 50: 9}
diameter_pairs = {5: (1, 5), 10: (1, 9), 15: (1, 15), 20: (11, 16),
                  30: (16, 23), 50: (32, 47)}

for N_val in sorted(diameter_claimed.keys()):
    # Compute actual diameter
    max_d = 0
    best_pair = (1, 1)
    for m in range(1, N_val + 1):
        for n in range(m + 1, N_val + 1):
            dn = d(m, n)
            if dn > max_d:
                max_d = dn
                best_pair = (m, n)

    claimed = diameter_claimed[N_val]
    CHECK(max_d == claimed,
          f"D({N_val}) = {claimed} (Table 2)",
          f"got D({N_val}) = {max_d}")

    # Check the claimed pair achieves it
    cp = diameter_pairs[N_val]
    CHECK(d(cp[0], cp[1]) == claimed,
          f"  achieved at d({cp[0]}, {cp[1]}) = {claimed}",
          f"d({cp[0]}, {cp[1]}) = {d(cp[0], cp[1])}")

    # Verify bounds
    logN = math.log(N_val)
    lower = c1 * logN - 1
    upper = c2 * logN
    CHECK(max_d >= lower,
          f"  D({N_val}) = {max_d} ≥ c_1·log({N_val}) - 1 = {lower:.2f}")
    CHECK(max_d <= upper,
          f"  D({N_val}) = {max_d} ≤ c_2·log({N_val}) = {upper:.2f}")

# Verify lower bound construction: D(N) ≥ d(1, 2^k) = k
print()
for k in range(1, 11):
    N_val = 2 ** k
    actual_d1 = d(1, N_val)
    CHECK(actual_d1 == k,
          f"d(1, 2^{k}) = d(1, {N_val}) = {k}",
          f"got {actual_d1}")


# ===========================================================================
# CELL 15: VERIFY PROPOSITION 7.1 — d(n, n+1) ≥ 2 for n ≥ 3
# ===========================================================================

SECTION("PROPOSITION 7.1: d(n, n+1) = 1 iff n ∈ {1, 2}")

CHECK(d(1, 2) == 1, "d(1, 2) = 1")
CHECK(d(2, 3) == 1, "d(2, 3) = 1")

# Verify d(n, n+1) ≥ 2 for n ≥ 3
consec_min = float('inf')
N_CONSEC = 1000
for n in range(3, N_CONSEC + 1):
    dn = d(n, n + 1)
    if dn < 2:
        CHECK(False, f"d(n, n+1) ≥ 2 for n ≥ 3",
              f"d({n}, {n+1}) = {dn}")
        break
    consec_min = min(consec_min, dn)
else:
    CHECK(True, f"d(n, n+1) ≥ 2 for all n in [3, {N_CONSEC}] "
          f"(minimum = {consec_min})")

# Verify d(n,n+1) = 1 does NOT occur for n ≥ 3
ones = [n for n in range(3, N_CONSEC + 1) if d(n, n + 1) == 1]
CHECK(len(ones) == 0,
      f"d(n, n+1) ≠ 1 for all n in [3, {N_CONSEC}]",
      f"d(n, n+1) = 1 at n ∈ {ones}")


# ===========================================================================
# CELL 16: VERIFY THEOREM 7.2 — Unboundedness of d(n, n+1)
# ===========================================================================

SECTION("THEOREM 7.2: d(2^k - 1, 2^k) → ∞")

# Table in the paper
paper_values = {
    2: 2, 3: 4, 4: 5, 5: 8, 6: 7, 7: 11, 8: 11, 9: 14, 10: 15
}

print("  k | 2^k  | f(2^k) | f(2^k-1) | d(2^k-1, 2^k) | paper")
print("  " + "-" * 60)

all_match = True
for k in range(2, 11):
    n = 2 ** k
    fn = f(n)
    fn_minus = f(n - 1)
    dist = d(n - 1, n)
    claimed = paper_values.get(k, "?")
    match = "✓" if dist == claimed else "✗"
    if dist != claimed:
        all_match = False
    print(f"  {k:2d} | {n:4d} | {fn:6d} | {fn_minus:8d} | {dist:14d} | {claimed} {match}")

CHECK(all_match, "All d(2^k - 1, 2^k) values match Table in Theorem 7.2")

# Verify T(2^k) is the star S_k (root + k leaves)
for k in range(1, 11):
    t = matula_tree(2 ** k)
    children = list(t)
    all_leaves = all(c == () for c in children)
    CHECK(len(children) == k and all_leaves,
          f"T(2^{k}) = S_{k} (star with {k} leaves)",
          f"got {len(children)} children, all leaves: {all_leaves}")

# Verify the sequence is unbounded (it should increase)
print()
prev_max = 0
is_unbounded = True
for k in range(2, 11):
    dist = d(2**k - 1, 2**k)
    if dist > prev_max:
        prev_max = dist
if prev_max >= 10:
    CHECK(True, f"d(2^k - 1, 2^k) reaches {prev_max} by k = 10, confirming growth")
else:
    WARN(f"Maximum d(2^k-1, 2^k) only reached {prev_max}")


# ===========================================================================
# CELL 17: VERIFY TABLE 3 — Record consecutive distances
# ===========================================================================

SECTION("TABLE 3: Record values of d(n, n+1)")

# Paper Table 3
records_claimed = [
    (1, 1), (3, 2), (4, 3), (7, 4), (15, 5),
    (16, 6), (31, 8), (64, 9), (96, 10), (127, 11)
]

# Compute actual records
record = 0
actual_records = []
for n in range(1, 130):
    dn = d(n, n + 1)
    if dn > record:
        record = dn
        actual_records.append((n, dn))

print("  Computed records:")
for n, dn in actual_records:
    print(f"    d({n}, {n+1}) = {dn}")
print()

records_ok = True
for claimed_n, claimed_d in records_claimed:
    actual_d = d(claimed_n, claimed_n + 1)
    if actual_d != claimed_d:
        CHECK(False, f"d({claimed_n}, {claimed_n + 1}) = {claimed_d}",
              f"got {actual_d}")
        records_ok = False
if records_ok:
    CHECK(True, "All record values in Table 3 are correct")

# Check that the records list is complete (no missing records)
CHECK(actual_records == records_claimed,
      "Record list is complete (no missing records up to n = 129)",
      f"computed: {actual_records}")


# ===========================================================================
# CELL 18: VERIFY SECTION 8 — Metric balls B(1, r)
# ===========================================================================

SECTION("PROPOSITION 8.1: Metric balls B(1, r)")

# Paper claims
balls_claimed = {
    0: [1],
    1: [1, 2],
    2: [1, 2, 3, 4],
    3: [1, 2, 3, 4, 5, 6, 7, 8],
    4: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 19],
}

# Compute actual balls
N_BALL = 500  # search up to this
for r in range(5):
    actual_ball = sorted([n for n in range(1, N_BALL + 1) if d(1, n) <= r])
    claimed = balls_claimed[r]
    CHECK(actual_ball == claimed,
          f"B(1, {r}) = {claimed}",
          f"got {actual_ball}")

# Check that B(1, r) = {n : f(n) ≤ r + 1}
for r in range(5):
    ball_via_d = sorted([n for n in range(1, N_BALL + 1) if d(1, n) <= r])
    ball_via_f = sorted([n for n in range(1, N_BALL + 1) if f(n) <= r + 1])
    CHECK(ball_via_d == ball_via_f,
          f"B(1, {r}) = {{n : f(n) ≤ {r + 1}}} (characterisation)")

# OEIS A000081 cumulative: number of rooted trees on ≤ k vertices
# A000081: 1, 1, 2, 4, 9, 20, 48, 115, 286, ...
# Cumulative: t(1)=1, t(2)=2, t(3)=4, t(4)=8, t(5)=17, ...
# So |B(1,r)| should be: 1, 2, 4, 8, 17
ball_sizes_expected = {0: 1, 1: 2, 2: 4, 3: 8, 4: 17}
for r, expected_size in ball_sizes_expected.items():
    actual_size = len(balls_claimed[r])
    CHECK(actual_size == expected_size,
          f"|B(1, {r})| = {expected_size} = t({r + 1})",
          f"got {actual_size}")

# Verify the remark about f(15), f(18), f(20) = 6
CHECK(f(15) == 6, "f(15) = 6 (as claimed in Remark)", f"got {f(15)}")
CHECK(f(18) == 6, "f(18) = 6 (as claimed in Remark)", f"got {f(18)}")
CHECK(f(20) == 6, "f(20) = 6 (as claimed in Remark)", f"got {f(20)}")


# ===========================================================================
# CELL 19: VERIFY SECTION 9 — Full distance table
# ===========================================================================

SECTION("TABLE 4: Distance table d(m, n) for 1 ≤ m, n ≤ 15")

# Paper's claimed table (row-major, 0s on diagonal)
paper_table = [
    [0,1,2,2,3,3,3,3,4,4,4,4,4,4,5],
    [1,0,1,1,2,2,2,2,3,3,3,3,3,3,4],
    [2,1,0,2,1,1,1,3,2,2,2,2,2,2,3],
    [2,1,2,0,3,1,3,1,2,2,4,2,4,2,3],
    [3,2,1,3,0,2,2,4,3,1,1,3,1,3,2],
    [3,2,1,1,2,0,2,2,1,1,3,1,3,1,2],
    [3,2,1,3,2,2,0,4,3,3,3,3,1,1,4],
    [3,2,3,1,4,2,4,0,3,3,5,1,5,3,4],
    [4,3,2,2,3,1,3,3,0,2,4,2,4,2,1],
    [4,3,2,2,1,1,3,3,2,0,2,2,2,2,1],
    [4,3,2,4,1,3,3,5,4,2,0,4,2,4,3],
    [4,3,2,2,3,1,3,1,2,2,4,0,4,2,3],
    [4,3,2,4,1,3,1,5,4,2,2,4,0,2,3],
    [4,3,2,2,3,1,1,3,2,2,4,2,2,0,3],
    [5,4,3,3,2,2,4,4,1,1,3,3,3,3,0],
]

table_ok = True
mismatches = []
for i in range(15):
    for j in range(15):
        m, n = i + 1, j + 1
        actual = d(m, n)
        claimed = paper_table[i][j]
        if actual != claimed:
            mismatches.append((m, n, claimed, actual))
            table_ok = False

CHECK(table_ok,
      f"All {15*15} = 225 entries of the 15×15 distance table are correct",
      f"{len(mismatches)} mismatches: {mismatches[:5]}")


# ===========================================================================
# CELL 20: VERIFY REMARK 9.1 — Table observations
# ===========================================================================

SECTION("REMARK 9.1: Observations about the distance table")

# (1) Column d(1, n) reads off f(n) - 1
obs1_ok = True
for n in range(1, 16):
    if d(1, n) != f(n) - 1:
        obs1_ok = False
CHECK(obs1_ok, "d(1, n) = f(n) - 1 for n = 1..15 (Observation 1)")

# (2) d(3, n) = d(2, i) when n = p_i
obs2_ok = True
for i in range(1, 8):
    n = nth_prime(i)
    if n <= 15:
        if d(3, n) != d(2, i):
            obs2_ok = False
CHECK(obs2_ok, "d(3, p_i) = d(2, i) for primes p_i ≤ 15 (Observation 2)")
CHECK(d(3, 5) == d(2, 3) == 1,
      "d(3, 5) = d(2, 3) = 1 (explicit example, since 5 = p_3)")

# (3) d(4, 9) = 2 < d(4, 5) = 3
CHECK(d(4, 9) == 2, "d(4, 9) = 2 (Observation 3)", f"got {d(4,9)}")
CHECK(d(4, 5) == 3, "d(4, 5) = 3 (Observation 3)", f"got {d(4,5)}")
CHECK(d(4, 9) < d(4, 5),
      "d(4, 9) < d(4, 5): multiplicative similarity beats Euclidean proximity")

# (4) d(2^a, 2^b) = |a - b| for powers of 2
pow2_ok = True
for a in range(1, 8):
    for b in range(a, 8):
        actual = d(2**a, 2**b)
        expected = abs(a - b)
        if actual != expected:
            pow2_ok = False
            CHECK(False, f"d(2^{a}, 2^{b}) = |{a}-{b}|",
                  f"d({2**a}, {2**b}) = {actual} ≠ {expected}")
if pow2_ok:
    CHECK(True, "d(2^a, 2^b) = |a - b| for 1 ≤ a ≤ b ≤ 7 (Observation 4)")

# Verify the claimed mechanism: γ(S_a, S_b) = 1 + min(a, b)
for a in range(1, 8):
    for b in range(a, 8):
        ta = matula_tree(2**a)
        tb = matula_tree(2**b)
        gamma = mces(ta, tb)
        expected_gamma = 1 + min(a, b)
        CHECK(gamma == expected_gamma,
              f"γ(S_{a}, S_{b}) = 1 + min({a},{b}) = {expected_gamma}",
              f"got {gamma}")


# ===========================================================================
# CELL 21: VERIFY CONJECTURE 10.6
# ===========================================================================

SECTION("CONJECTURE 10.6: d(2^k - 1, 2^k) ≥ k for k ≥ 2")

conj_ok = True
for k in range(2, 11):
    dist = d(2**k - 1, 2**k)
    if dist < k:
        CHECK(False, f"d(2^{k} - 1, 2^{k}) ≥ {k}",
              f"d({2**k - 1}, {2**k}) = {dist}")
        conj_ok = False
    else:
        CHECK(True, f"d(2^{k} - 1, 2^{k}) = {dist} ≥ {k}")


# ===========================================================================
# CELL 22: ADDITIONAL CROSS-CHECKS
# ===========================================================================

SECTION("ADDITIONAL CROSS-CHECKS")

# (a) T is a bijection on small integers: distinct n give distinct trees
trees_seen = {}
bijection_ok = True
for n in range(1, 201):
    t = matula_tree(n)
    if t in trees_seen:
        CHECK(False, "T is injective",
              f"T({n}) = T({trees_seen[t]})")
        bijection_ok = False
        break
    trees_seen[t] = n
if bijection_ok:
    CHECK(True, "T is injective on [1, 200] (200 distinct trees)")

# (b) Verify the recursion γ(S, T) = 1 + max matching matches
#     our brute-force d computation for random pairs
import random
random.seed(42)
consistency_ok = True
for _ in range(200):
    m = random.randint(1, 100)
    n = random.randint(1, 100)
    tm = matula_tree(m)
    tn = matula_tree(n)
    d_formula = tree_size(tm) + tree_size(tn) - 2 * mces(tm, tn)
    d_direct = ted(tm, tn)
    if d_formula != d_direct:
        consistency_ok = False
        break
CHECK(consistency_ok,
      "δ = |S| + |T| - 2γ consistent with cached ted() for 200 random pairs")

# (c) Verify planted tree structure for primes
for k in range(1, 20):
    p = nth_prime(k)
    t = matula_tree(p)
    # T(p_k) should be planted: root of degree 1 whose child is T(k)
    CHECK(len(t) == 1,
          f"T(p_{k}) = T({p}) is planted (root degree 1)",
          f"root degree = {len(t)}")
    if len(t) == 1:
        CHECK(t[0] == matula_tree(k),
              f"  child subtree of T({p}) is T({k})")

# (d) Verify f(n) for n = 1..20 against the computation output
f_values = {
    1: 1, 2: 2, 3: 3, 4: 3, 5: 4, 6: 4, 7: 4, 8: 4, 9: 5, 10: 5,
    11: 5, 12: 5, 13: 5, 14: 5, 15: 6, 16: 5, 17: 5, 18: 6, 19: 5, 20: 6
}
for n, expected in f_values.items():
    CHECK(f(n) == expected, f"f({n}) = {expected}", f"got {f(n)}")


# ===========================================================================
# CELL 23: STRESS TEST — Extended range checks
# ===========================================================================

SECTION("STRESS TESTS")

# (a) Isometry for larger indices
print("  Extended isometry check (i, j up to 100)...")
iso_extended_ok = True
iso_extended_count = 0
for i in range(1, 30):
    for j in range(i + 1, 30):
        pi = nth_prime(i)
        pj = nth_prime(j)
        if pi <= 500 and pj <= 500:
            d_p = d(pi, pj)
            d_ij = d(i, j)
            if d_p != d_ij:
                iso_extended_ok = False
                print(f"    FAIL: d(p_{i}, p_{j}) = d({pi},{pj}) = {d_p} "
                      f"≠ d({i},{j}) = {d_ij}")
                break
            iso_extended_count += 1
    if not iso_extended_ok:
        break
CHECK(iso_extended_ok,
      f"Extended isometry check passed ({iso_extended_count} pairs)")

# (b) Bound f(n) ≤ 1 + α log n for n up to 50000
print("  Extended vertex bound check (n up to 50000)...")
ext_bound_ok = True
for n in range(2, 50001):
    fn = f(n)
    bound_val = 1 + alpha * math.log(n)
    if fn > bound_val + 1e-9:
        ext_bound_ok = False
        print(f"    FAIL: f({n}) = {fn} > {bound_val:.4f}")
        break
CHECK(ext_bound_ok, "f(n) ≤ 1 + α log n for all n in [2, 50000]")

# (c) d(n, n+1) ≥ 2 for n in [3, 2000]
print("  Extended consecutive distance check (n up to 2000)...")
ext_consec_ok = True
for n in range(3, 2001):
    if d(n, n + 1) < 2:
        ext_consec_ok = False
        print(f"    FAIL: d({n}, {n+1}) = {d(n, n+1)}")
        break
CHECK(ext_consec_ok, "d(n, n+1) ≥ 2 for all n in [3, 2000]")


# ===========================================================================
# CELL 24: SUMMARY
# ===========================================================================

SECTION("FINAL SUMMARY")

print(f"  Passed:   {_pass_count}")
print(f"  Failed:   {_fail_count}")
print(f"  Warnings: {_warn_count}")
print()

if _fail_count == 0:
    print("  ★ ALL CHECKS PASSED ★")
    print()
    print("  Every numerically verifiable claim in the paper has been")
    print("  independently confirmed.")
else:
    print(f"  ✗ {_fail_count} CHECK(S) FAILED — see details above.")
    print()
    print("  The paper contains errors that must be corrected.")

print()
print("=" * 72)
print("END OF VERIFICATION")
print("=" * 72)


VERIFICATION SCRIPT
'The Matula metric on the natural numbers' — S. Severini


------------------------------------------------------------------------
  EXAMPLE 2.1: Matula trees T(n) for n = 1, ..., 8
------------------------------------------------------------------------
  ✓ |T(1)| = 1 (single vertex)
  ✓ |T(2)| = 2 (root + one leaf)
  ✓ |T(3)| = 3 (path of length 2)
  ✓ |T(4)| = 3 (root + two leaves (star S_2))
  ✓ |T(5)| = 4 (path of length 3)
  ✓ |T(6)| = 4 (T(1) and T(2) attached to root)
  ✓ |T(7)| = 4 (root + star S_2)
  ✓ |T(8)| = 4 (root + three leaves (star S_3))
  ✓ T(1) is single vertex ()
  ✓ T(2) = root + one leaf
  ✓ T(3) = path: root → child → leaf
  ✓ T(4) = root + two leaves
  ✓ T(5) = path of 4 vertices
  ✓ T(8) = root + three leaves (star)
  ✓ T(6) = root with T(1) and T(2) as children
  ✓ T(7) = planted S_2

------------------------------------------------------------------------
  PROPOSITION 3.1: d is a metric on N
---------------------------------------------